In [ ]:
# Only if you are on Google Colab, run this and the next two cells
# This will crash your kernel- this is OK and expected! Run this just once,
# then the next two cells 
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#Google Colab status check
import condacolab
condacolab.check()

In [ ]:
#Google Colab software stack install
!mamba install -c conda-forge "hoomd=4.7.0" vim openssh mbuild gsd ipython ipykernel mdanalysis foyer freud ipywidgets scipy numpy physical_validation matplotlib jupyter pandas flowermd nglview mdanalysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import freud
import gsd.hoomd

In [ ]:
def basic_inline_visual(filename, frame):
    with gsd.hoomd.open(filename) as traj:
        frame = traj[frame]

        figure = plt.figure(figsize=(10,10))
        ax = figure.add_subplot(projection='3d')
        pos = frame.particles.position
        
        ax.scatter(pos[:,0],pos[:,1], pos[:,2])

# What is a Radial Distribution Function?

One of the main tools of simulations is the Radial Distribution Function, or RDF. In physical experiments, it's often difficult to obtain an RDF, so we use reciprocal space techniques like XRD or do laborious microscopy. In simulations we know the exact position of every particle, and so building an RDF is relatively easy.

Here we will walk through 3 exercises.

1. Calculating a small RDF by hand
2. Using a histogram to calculate a larger RDF
3. Using the tool Freud to examine your LJ simulations and compare how changing temperature changes RDF

You may find the Freud documentation useful while doing excersizes in this notebook:
https://freud.readthedocs.io/en/latest/modules/density.html

$$g(r) = V \frac{N_{\text{reference particles}}}{N_{\text{particles}}} \langle\delta(r)\rangle$$


The Radial Distribution Function is usually denoted g(r). It calculates the average density $\delta$ of particles at a distance *r* away from an origin particle, normalized by the volume. If the system is totally homogenous and disordered, g(r) = 1. Typically, this function will be averaged over every particle in a system, and over time. From the RDF, we can extract the distance between particles and understand how well ordered the system is.

The following code will plot an ideal square 2D crystal with some amount of noise, or disorder, in the position of the particles. 0 noise indicates a perfect crystal.

Change the noise level and observe how the RDF changes. While the physics are very different, larger noise is qualitatively similar to melting the crystal.

In [ ]:
#noise 0.0-0.2 is a good range to explore.
noise = 0.0

In [ ]:
crystal_with_disorder = freud.data.UnitCell([1,1,1,0,0,0],[[0,0,0]]).generate_system(
    num_replicas = 20, sigma_noise = noise)

In [ ]:
rdf = freud.density.RDF(bins = 100, r_max = 3.0)
rdf.compute(crystal_with_disorder,neighbors={"r_max":3.0,"exclude_ii":True})
rdf.plot()

What does the RDF look like in a perfectly ordered system? What happens to the RDF as disorder increases?

# Manual RDF

Now we are going to do a quick, rough calculation of the RDF for one particle to get an idea of how the algorithm works. This code will generate a small 2D system of particles.

In [ ]:
(box, positions) = freud.data.UnitCell([1,1,1,0,0,0],[[0,0,0]]).generate_system(
    num_replicas = [10,10,1], sigma_noise = 0.1)

In [ ]:
center = positions[0]

In [ ]:
shell_radii = np.arange(0.0,4.0,0.5)

In [ ]:
fig = plt.figure(figsize=(10,10))
markersize = 150

plt.scatter(positions[:,0],positions[:,1],s = markersize, c = 'grey',edgecolor = 'k')
plt.scatter(center[0],center[1], s = markersize, c = 'red',edgecolor = 'red')

for r in shell_radii:
    circle = plt.Circle((0, 0), r, facecolor = [0,0,0,0], edgecolor = 'k', linewidth = 2)
    plt.gca().add_patch(circle)
    
plt.xlim(-5,5);
plt.ylim(-5,5);


Take the origin to be at [0,0]. The circular shells have radius 1, 2, 3, and 4. Working with your neighbor, count the number particles within the bounds of each shell. You may have to make some judgement calls if a particle sits close to a line.

The system has an overall number density of 50 particles in a 10x10 area.

*Commented out are example numbers if you need them, for an ideal gas system*

In [ ]:
# particle_counts = ?
particle_counts = [0, 5, 6, 13, 20, 25,30]

Now change the particle counts to a density by calculating the *area* of each shell

In [ ]:
areas = [np.pi*((shell_radii[n+1])**2-shell_radii[n]**2) for n in range(7)]
particle_density = np.asarray(particle_counts)/np.asarray(areas)

In [ ]:
V = box.volume
N = len(positions)
gofr = V * 1/N * particle_density

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(shell_radii[1:],gofr);

plt.title('Manual count of a single-particle RDF')
plt.xlabel('Distance')
plt.ylabel('Particle density')
plt.xlim(0,4);

This system isn't large enough for this RDF to look much like the example. Notably, the width of our "bins" is extremely large, so a lot of the shape is washed out. Hopefully you now have an idea of how the algorithm works.

# Calculating RDF Using a Histogram

Now we'll use a histogram to calculate a much more realistic RDF. 

In [ ]:
(box, positions) = freud.data.UnitCell([1,1,1,0,0,0],[[0,0,0]]).generate_system(
    num_replicas = [20,20,1], sigma_noise = 0.05)

In [ ]:
center = positions[0]

In [ ]:
fig = plt.figure(figsize=(10,10))
markersize = 150

plt.scatter(positions[:,0],positions[:,1],s = markersize, c = 'grey',edgecolor = 'k')
plt.scatter(center[0],center[1], s = markersize, c = 'red',edgecolor = 'red')

for r in shell_radii:
    circle = plt.Circle((0, 0), r, facecolor = [0,0,0,0], edgecolor = 'k', linewidth = 2)
    plt.gca().add_patch(circle)
    
plt.xlim(-10,10);
plt.ylim(-10,10);


For now, only calculate one particle (no averaging).

How are you going to calculate the density?

In [ ]:
distances = []

for p in positions[1:]:
    d = np.linalg.norm(p-center)
    distances.append(d)

bins = 50
histogram, bin_edges = np.histogram(distances, range = (0.0,5.0), bins = bins)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(bin_edges[:-1],histogram)

plt.xlabel('Distance')
plt.ylabel('Particle Count')
plt.xlim(0,5);

Change bin_edges to a list of areas to get density, using the same method that you did during the manual count.

In [ ]:
areas = [np.pi*((bin_edges[n+1])**2-bin_edges[n]**2) for n in range(bins)]
delta = histogram/areas

In [ ]:
V = box.volume
N = len(positions)
gofr = V * 1/N * delta

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(bin_edges[:-1],gofr);

plt.title('Programmatic count of a single-particle RDF')
plt.xlabel('Distance')
plt.ylabel('g(r)')
plt.xlim(0,5);


Again, this system is still not large enough to really see the expected form of an RDF, but hopefully you get the idea.

# Questions

1. Where does it make sense to stop this histogram? At what point do you need to consider boundary conditions?

2. How does changing the number of bins affect the calculated RDF?

3. A simple loop algorithm to calculate neigbhors quickly becomes inefficient for systems of thousands of particles. Can you think of a way to increase speed?

# Large LJ System

Now we can load the Lennard-Jones fluids generated in previous simulations for analysis. We're going to use the Freud utility to handle many of the difficult calculations for us, including accounting for periodic boundary conditions and making the math efficient when dealing with thousands of particles instead of dozens.

Play here with n_bins and r_max. How does changing n_bins effect your result? Is there an upper limit beyond which the width of your bins is too small?

In [ ]:
with gsd.hoomd.open('example_results/trajectory.gsd','r') as traj:
    final_frame = traj[-1]
    
box = final_frame.configuration.box
positions = final_frame.particles.position

In [ ]:
n_bins = 100
r_max = 4.0

rdf = freud.density.RDF(bins = n_bins, r_max = r_max)
rdf.compute((box, positions))
rdf.plot()

In the last section, you and your neighbors simulated fluids at different conditions, but the same *reduced unit* conditions. Compare your results; how does the RDF change? 

How does the fluid RDF compare to the idealized crystal in the example?

# Averaging for better statistics

Finally, to be very correct we should also *time*-average the RDF. Keeping in mind that "traj" is a list of all frames in the simulation, how do you calculate the average RDF?

*Hint: you will need to loop over every frame in the trajectory, saving the RDF at each loop to average later.*

*Additional hint: freud.density.RDF.compute() has a "reset" property, which you can turn to False.*

The function rdf.plot() which we used as a shortcut above is essentially running the following function:

plt.plot(rdf.bin_centers, rdf.rdf)

In [ ]:
n_bins = 100
r_max = 4.0
rdf = freud.density.RDF(bins = n_bins, r_max = r_max)

with gsd.hoomd.open('example_results/trajectory.gsd','r') as trajectory:
    #loop over all frames:
    for frame in trajectory:
        #accumulate an average RDF
        box = frame.configuration.box
        positions = frame.particles.position
        rdf.compute((box, positions), reset = False)
        


In [ ]:
plt.figure(figsize=(5,5))

plt.plot(rdf.bin_centers, rdf.rdf)

plt.title('Time-Averaged RDF')
plt.xlabel('Distance')
plt.ylabel('g(r)')

How does performing a time average change the RDF?